In [1]:
import sys
print(sys.version)

import tensorflow as tf
print(tf.__version__)

import numpy as np
np.set_printoptions(threshold=np.inf)

3.8.15 | packaged by conda-forge | (default, Nov 22 2022, 08:52:09) 
[Clang 14.0.6 ]
2.12.0


In [3]:
"""
1. metrics 性能指标
加权平均值： tf.keras.metrics.Mean
预测值和真实值的准确度： tf.keras.metrics.Accuracy

1.1 新建一个metrics指标
准确度指标 metrics.Accuracy() 一般用于训练集，加权平均值 metrics.Mean()  一般用于测试集
"""
from tensorflow.keras import metrics
# 新建准确度指标
acc_meter = metrics.Accuracy() 
print(acc_meter)
# 新建平均值指标
mean_meter = metrics.Mean()  
print(mean_meter)

Accuracy(name=accuracy,dtype=float32)
Mean(name=mean,dtype=float32)


In [ ]:
"""
1.2 向metrics添加数据
添加数据：update_state()。每一次迭代，都向准确率指标中添加测试数据的真实值和测试数据的预测值，将准确率保存在缓存区，需要时取出来。
向平均损失指标中添加每一次训练产生的损失，每添加进来一个值就计算加权平均值，
sample_weight指定每一项的权重，将结果保存在缓存区，需要时取出来。
"""
# 计算真实值和预测值之间的准确度
acc_meter.update_state(y_true, predict) 
# 计算平均损失
mean_meter = mean_meter.update_state(loss, sample_weight=None)

In [ ]:
"""
1.3 从metrics中取出数据
取出数据：result().numpy()。result()返回tensor类型数据，转换成numpy()类型的数据。
"""
# 取出准确率
acc_meter.result().numpy() 
# 取出训练集的损失值的均值
mean_meter.result().numpy()

In [ ]:
"""
清空缓存：reset_states()。每一次循环缓存区都会将之前的数据保存，在开始第二次循环之前，应该把缓存区清空，重新读入数据。
"""
# 清空准确率的缓存
acc_meter.reset_states()
# 清空加权均值的缓存
mean_meter.reset_states()

In [ ]:
"""
2. compile 模型配置
compile(optimizer, loss, metrics, loss_weights)
参数设置：
optimizer： 用来配置模型的优化器，可以调用tf.keras.optimizers API配置模型所需要的优化器。
loss： 用来配置模型的损失函数，可以通过名称调用tf.losses API中已经定义好的loss函数。
metrics： 用来配置模型评价的方法，模型训练和测试过程中的度量指标，如accuracy、mse等
loss_weights： float类型，损失加权系数，总损失是所有损失的加权和，它的元素个数和模型的输出数量是1比1的关系。
"""
# 选择优化器Adam，loss为交叉熵损失，测试集评价指标accurancy
network.compile(optimizer=optimizers.legacy.Adam(learning_rate=0.01) , #学习率0.01
    loss = tf.losses.CategoricalCrossentropy(from_logits=True),
    metrics = ['accuracy'])

In [ ]:
"""
3. fit 模型训练
fit(x, y, batch_size, epochs, validation_split, validation_data, shuffle, validation_freq)
参数：
x： 训练集的输入数据，可以是array或者tensor类型。
y： 训练集的目标数据，可以是array或者tensor类型。
batch_size：每一个batch的大小，默认32
epochs： 迭代次数
validation_split：配置测试集数据占训练数据集的比例，取值范围为0～1。
validation_data： 配置测试集数据(输入特征及目标)。如果已经配置validation_split参数，则可以不配置该参数。
    如果同时配置validation_split和validation_data参数，那么validation_split参数的配置将会失效。
shuffle：配置是否随机打乱训练数据。当配置steps_per_epoch为None时，本参数的配置失效。
validation_freq： 每多少次循环做一次测试
"""
# ds为包含输入特征及目标的数据集
network.fit(ds, eopchs=20, validation_data=ds_val, validation_freq=2)
# validation_data给定测试集，validation_freq每多少次大循环做一次测试，测试时自动计算准确率

In [ ]:
"""
4. evaluate 模型评估
evaluate(x, y, batch_size, sample_weight, steps)
返回模型的损失及准确率等相关指标

参数：
x： 输入测试集特征数据
y：测试集的目标数据
batch_size： 整数或None。每个梯度更新的样本数。如果未指定，batch_size将默认为32。如果数据采用数据集，生成器形式，则不要指定batch_size。
sample_weight： 测试样本的可选Numpy权重数组，用于加权损失函数。
steps： 整数或None。宣布评估阶段结束之前的步骤总数。
"""

In [ ]:
"""
5. predict 预测
predict(x, batch_size, steps)
参数：

x： numpy类型，tensor类型。预测所需的特征数据
batch_size： 每个梯度更新的样本数。如果未指定，batch_size将默认为32
steps： 整数或None，宣布预测回合完成之前的步骤总数（样本批次）。
等同于：
"""
sample = next(iter(ds_pred)) # 每次从验证数据中取出一组batch
x = sample[0] # x 保存第0组验证集特征值
pred = network.predict(x)  # 获取每一个分类的预测结果
pred = tf.argmax(pred, axis=1) # 获取值最大的所在的下标即预测分类的结果
print(pred)

In [ ]:
"""
6. sequential
Sequential模型适用于简单堆叠网络层，即每一层只有一个输入和一个输出。
"""
# ==1== 设置全连接层
# [b,784]=>[b,256]=>[b,128]=>[b,64]=>[b,32]=>[b,10]，中间层一般从大到小降维
network = Sequential([
    layers.Dense(256, activation='relu'), #第一个连接层，输出256个特征
    layers.Dense(128, activation='relu'), #第二个连接层
    layers.Dense(64, activation='relu'), #第三个连接层
    layers.Dense(32, activation='relu'), #第四个连接层
    layers.Dense(10), #最后一层不需要激活函数，输出10个分类
    ])
# ==2== 设置输入层维度
network.build(input_shape=[None, 28*28])
# ==3== 查看网络结构
network.summary()
# ==4== 查看网络的所有权重和偏置
network.trainable_variables
# ==5== 自动把x从第一层传到最后一层
network.call()

In [ ]:
"""
7. 自定义层构建网络
通过对 tf.keras.Model 进行子类化并定义自己的前向传播模型。在 __init__ 方法中创建层并将它们设置为类实例的属性。在 call 方法中定义前向传播。
"""
# 自定义Dense层
class MyDense(layers.Layer): #必须继承layers.Layer层，放到sequential容器中
    # 初始化方法
    def __int__(self, input_dim, output_dim):
        super(MyDense, self).__init__() # 调用母类初始化，必须
        
        # 自己发挥'w''b'指定名字没什么用，创建shape为[input_dim, output_dim的权重
        # 使用add_variable创建变量
        self.kernel = self.add_variable('w', [input_dim, output_dim])
        self.bias = self.add_variable('b', [output_dim])
    
    # call方法，training来指示现在是训练还是测试
    def call(self, inputs, training=None):
        out = inputs @ self.kernel + self.bias
        return out
 
 
# 自定义层来创建网络
class MyModel(keras.Model):  # 必须继承keras.Model大类，才能使用complie、fit等功能
    # 
    def __init__(self):
        super(MyModel, self).__init__() # 调用父类Mymodel
        # 使用自定义层创建5层
        self.fc1 = MyDense(28*28,256) #input_dim=784，output_dim=256
        self.fc2 = MyDense(256,128)
        self.fc3 = MyDense(128,64)
        self.fc4 = MyDense(64,32)
        self.fc5 = MyDense(32,10)
 
    def call(self, inputs, training=None):
        # x从输入层到输出层
        x = self.fc1(inputs)
        x = tf.nn.relu(x)
        x = self.fc2(x)
        x = tf.nn.relu(x)        
        x = self.fc3(x)
        x = tf.nn.relu(x)
        x = self.fc4(x)
        x = tf.nn.relu(x)
        x = self.fc5(x) #logits层
        return x